In [ ]:
!pip install transformers datasets sentencepiece rouge -qq

In [ ]:
import torch
import pandas as pd
from rouge import Rouge
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [ ]:
rouge = Rouge()
dataset = load_dataset("d0r1h/ILC", split='test')s

In [ ]:
CasesText = dataset['Case']
GoldSummary = dataset['Summary']

In [ ]:
len(CasesText), len(GoldSummary)

(1015, 1015)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
def summarize(model, tokenizer, Cases):

  SystemSummaries = []
  for i, case in enumerate(Cases):

      input_ids = tokenizer(case, return_tensors="pt").input_ids.to(device)
      global_attention_mask = torch.zeros_like(input_ids)
      global_attention_mask[:, 0] = 1
      sequences = model.generate(input_ids, global_attention_mask=global_attention_mask)
      Summary = tokenizer.batch_decode(sequences, skip_special_tokens=True)

      SystemSummaries.append(Summary)
      print(i)

  return SystemSummaries

In [ ]:
checkpoint = "d0r1h/led-base-ilc"

tokenizer_led = AutoTokenizer.from_pretrained(checkpoint)
model_led = AutoModelForSeq2SeqLM.from_pretrained(checkpoint).to(device)

In [ ]:
SystemSummary = summarize(model_led, tokenizer_led, CasesText)

In [ ]:
SystemSummaryFinal = []

for i in SystemSummary:
  SystemSummaryFinal.append((i[0]))

In [ ]:
Summaries = pd.DataFrame(list(zip(GoldSummary, SystemSummaryFinal)), columns =['GoldSummary', 'SystemSummary'])

In [ ]:
dir_path = "/content/drive/MyDrive/Working | Project/ILC/data/"

In [ ]:
df1 = pd.read_csv(dir_path + "Summaries1.csv")
df2 = pd.read_csv(dir_path + "Summaries2.csv")
df3 = pd.read_csv(dir_path + "Summaries3.csv")
df4 = pd.read_csv(dir_path + "Summaries4.csv")

In [ ]:
DF = pd.concat([df1, df2, df3, df4])
DF.reset_index(inplace=True, drop=True)

In [ ]:
DF

,GoldSummary,SystemSummary
0,Section 86(1)(f) vests a statutory jurisdictio...,In the case of Gujarat Urja Vikas Nigam Limite...
1,The petitioner apprehended arrest under Sectio...,The case was taken up out of turn on the basis...
2,The petitioner was arrested under Sections 344...,The petitioner is running Dance Party from man...
3,In matters concerning administrative appointme...,The petitioners were appointed as Inspectors i...
4,The facts and information of the suspected off...,The FIR was registered on the basis of a repor...
...,...,...
1010,"In the present case, an appeal is preferred un...",The court observed that the court has not inve...
1011,Re-evaluation of answer sheets is not permissi...,“If an error is committed by the examination a...
1012,The presence of an alternate land that can be ...,The Landlord has a registered Society under th...
1013,Bail may be granted to an individual who is ac...,"The applicant is arrested on 18th November, 20..."


In [ ]:
score = rouge.get_scores(DF['SystemSummary'], DF['GoldSummary'], avg=True)

In [ ]:
LEDRouge = pd.DataFrame(score).set_index([['recaall','precision','f-measure']])

In [ ]:
LEDRouge*100

,rouge-1,rouge-2,rouge-l
recaall,40.039816,22.343800,37.250347
precision,46.313938,25.252847,43.097858
f-measure,42.240362,23.187177,39.304978


In [ ]:
DF.to_csv(dir_path + "LEDPrediction.csv", index=False, header=True)

In [ ]:
LEDRouge.to_csv("/content/drive/MyDrive/Working | Project/ILC/score/LEDRouge.csv", header=True)